<h1>Get the number of pages</h1>

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import re
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import to_date,col,date_format,sort_array
from datetime import datetime
import os
from dotenv import load_dotenv
import logging

In [2]:
logging.basicConfig(
    filename='log.txt',     # Nom du fichier de log
    level=logging.ERROR,    # Niveau de gravité à capturer
    format='%(asctime)s - %(levelname)s - %(message)s',  # Format des messages
    filemode='a'            # Mode d'ouverture du fichier (ajout)
)

<p>We set up the url of our page. we can change the years of game releases, as well as their platform and many other criteria</p>

In [3]:
url = f'https://www.metacritic.com/browse/game/?releaseYearMin=2019&releaseYearMax=2024&page='

In [4]:
def get_number_pages(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url+"1", headers=headers)
    
    # if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        game_presence = soup.find('div',class_='c-finderProductCard')
        if game_presence is not None:
            try:
                number_page = soup.find('span',class_='c-navigationPagination_pages').find_all('span','c-navigationPagination_itemButtonContent')[-1]
                number_page = number_page.text.strip()
                print(number_page,"pages trouvées")
                return int(number_page)
            except Exception as e:
                print("1 pages trouvée")
                return 1
        else:
            print("Aucun jeu trouvé")
            return -1
    else:
        print(f"Erreur {response.status_code}")
        return -1

In [5]:
number_page = get_number_pages(url)

123 pages trouvées


<h1>Get the content of each page</h1>

In [6]:
#Function that, for a page number passed as parameter, will retrieve the content of the associated page
def fetch_page_data(numpage):
    time.sleep(2)
    if numpage > 0:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        try:
            response = requests.get(url+str(numpage), headers=headers, timeout=10)
            response.raise_for_status()  # Lève une exception si le code de statut n'est pas 200
            return numpage, response.text
        except HTTPError as http_err:
            print(f"Erreur HTTP pour la page {numpage}: {http_err}")
        except Timeout:
            print(f"Timeout pour la page {numpage}")
        except RequestException as req_err:
            print(f"Erreur de requête pour la page {numpage}: {req_err}")
    return numpage, "NULL"  # Si une erreur survient, retournez "NULL"

In [7]:
def create_tab_request(number_page):
    if number_page>0:
        html_content_tab_request = []
        # Using ThreadPoolExecutor for multithreading to speed up the process
        # For each page number from 1 to number_page, we call the fetch_page_data function and add the result in our result table
        with ThreadPoolExecutor(max_workers=20) as executor:
            # Submit tasks for each page
            futures = [executor.submit(fetch_page_data, numpage) for numpage in range(1, int(number_page) + 1)]
        
            # Wait for each task to be completed and retrieve the results
            for future in as_completed(futures):
                numpage, content = future.result()  # Get page number and content
                html_content_tab_request.append(content)
        return html_content_tab_request
    else:
        print("Number page negatif")
        return []

In [8]:
html_content_tab_request = create_tab_request(number_page)

<h1>Get game titles</h1>

In [9]:
#we get the info from all games
def get_game_titles(html_content_tab_request):
    titles = []
    if len(html_content_tab_request) > 0 and html_content_tab_request is not None:
        for html_page in html_content_tab_request:
            soup = BeautifulSoup(html_page, 'html.parser')
        
            links = soup.find_all('a',class_='c-finderProductCard_container')
            
            for element in links:
                title_no_transform = element.get('href')
                parts = title_no_transform.split("/")
                truncated_string = parts[2]
                titles.append(truncated_string)
    else:
        print("le tableau passé en paramètre est vide")
        
    return titles

In [10]:
titles = get_game_titles(html_content_tab_request)

In [11]:
print(len(titles))

2948


<h1>Get game info</h1>

In [12]:
def get_game_info(title):
    time.sleep(2)
    url = f'https://www.metacritic.com/game/{title}/'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # Récupérer la page
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Page inconnue pour {title}")
        return create_error_response(title)

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Retrieve game data
    metascore = get_metascore(title,soup)
    userscore = get_userscore(title,soup)
    reviews_number = get_reviews_number(title,soup)
    release_date = get_release_date(title,soup)
    genre = get_genre(title,soup)
    developer = get_developer(title,soup)
    publisher = get_publisher(title,soup)
    platforms = get_platforms(title,soup)

    return [title, metascore, userscore, reviews_number, release_date, genre, developer,publisher, platforms]

In [13]:
def create_error_response(title):
    """Renvoie une liste avec des valeurs d'erreur pour le jeu."""
    return [title, "-1", "-1", "-1","NULL", "NULL", "NULL", "NULL", ["NULL"]]

In [14]:
def get_metascore(title,soup):
    """Récupère le metascore du jeu."""
    try:
        metascore = soup.find('div', class_='c-siteReviewScore').find('span')
        return metascore.text
    except Exception as e:
        logging.error(f"Game : {title} - Erreur metascore inexistantes: {e}")
        return "-1"

In [15]:
def get_userscore(title,soup):
    """Récupère le score utilisateur du jeu."""
    try:
        userscore = soup.find('div', class_='c-siteReviewScore_user').find('span')
        #if the game doesn't have userscore ("tbd" value)
        if userscore.text=="tbd":
            return -1
        else:
            return userscore.text
    except Exception as e:
        #if the game doesn't have userscore (div userscore not exist)
        logging.error(f"Game : {title} - Erreur userscore inexistantes: {e}")
        return "-1"

In [16]:
def get_reviews_number(title,soup):
    """Récupère le nombre de critiques utilisateurs."""
    try:
        reviews_number = soup.find_all('span', class_='c-productScoreInfo_reviewsTotal')[1].find('span').text
        # Extract only the numbers
        match = re.search(r'\d+', reviews_number.replace(',', ''))
        return match.group(0) if match else "NULL"
    except Exception as e:
        logging.error(f"Game : {title} - Aucun avis utilisateur: {e}")
        return "0"

In [17]:
def get_release_date(title,soup):
    """Récupère la date de sortie du jeu."""
    try:
        release_date = soup.find_all('div', class_='g-text-xsmall')[1].find('span', class_='u-text-uppercase')
        return release_date.text if release_date else "error"
    except Exception as e:
        logging.error(f"Game : {title} - Erreur date inexistantes: {e}")
        return "NULL"

In [18]:
def get_genre(title,soup):
    """Récupère le genre du jeu."""
    try:
        genre = soup.find('li', class_='c-genreList_item').find('span', class_='c-globalButton_label')
        return genre.text.strip() if genre else "error"
    except Exception as e:
        logging.error(f"Game : {title} - Erreur genre inexistantes: {e}")
        return "NULL"

In [19]:
def get_developer(title,soup):
    """Récupère le développeur du jeu."""
    try:
        developer = soup.find('div', class_='c-gameDetails_Developer').find('li', class_='c-gameDetails_listItem')
        return developer.text.strip() if developer else "error"
    except Exception as e:
        logging.error(f"Game : {title} - Erreur developer inexistantes: {e}")
        return "NULL"


In [20]:
def get_publisher(title,soup):
    """Récupère l'éditeur du jeu."""
    try:
        publisher = soup.find('div', class_='c-gameDetails_Distributor').find_all('span')[1]
        return publisher.text.strip() if publisher else "error"
    except Exception as e:
        logging.error(f"Game : {title} - Erreur publisher inexistantes: {e}")
        return "NULL"


In [21]:
def get_platforms(title,soup):
    """Récupère les plateformes sur lesquelles le jeu est disponible."""
    try:
        platforms_list = soup.find('div', class_='c-gameDetails_Platforms').find_all('li')
        return [platform.text.strip() for platform in platforms_list] if platforms_list else ["error"]
    except Exception as e:
        logging.error(f"Game : {title} - Erreur plateforms inexistantes: {e}")
        return ["NULL"]

In [22]:
def fetch_all_games_info(titles):
    game_info_list = []

    # Using ThreadPoolExecutor to run in parallel
    with ThreadPoolExecutor(max_workers=30) as executor:
        # Submit tasks to be performed for each title
        future_to_game_info = {executor.submit(get_game_info, title): title for title in titles}

        # Collect results as soon as they are ready
        for future in as_completed(future_to_game_info):
            game_info = future_to_game_info[future]
            try:
                data = future.result()
                if data != "error":
                    game_info_list.append(data)
            except Exception as e:
                logging.error(f"{game_info} generated an exception: {e}")
    return game_info_list

In [23]:
all_game_infos = fetch_all_games_info(titles)

<h1>Data cleaning</h1>

In [24]:
load_dotenv()

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

conf = (
    SparkConf()
    .setAppName("MY_APP") # replace with your desired name
    .set("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0,org.apache.hadoop:hadoop-aws:3.3.2")
    .set("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .set("spark.hadoop.fs.s3a.access.key", aws_access_key_id)
    .set("spark.hadoop.fs.s3a.secret.key", aws_secret_access_key)
    .set("spark.sql.shuffle.partitions", "10")
    .setMaster("local[*]")
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

24/09/12 00:40:52 WARN Utils: Your hostname, MacBook-Air-de-Geoffrey.local resolves to a loopback address: 127.0.0.1; using 10.245.55.173 instead (on interface en0)
24/09/12 00:40:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/gvillevy/.ivy2/cache
The jars for the packages stored in: /Users/gvillevy/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1528b051-d869-4760-98d6-2a93042cf3a7;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/gvillevy/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 151ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------

In [25]:
#We create our dataframe with our information
df = spark.createDataFrame(all_game_infos, ["title","metascore","userscore","reviews_number","date","genre","developer","publisher","plateforms"])
df.show()

+--------------------+---------+---------+--------------+------------+----------------+--------------------+--------------------+--------------------+
|               title|metascore|userscore|reviews_number|        date|           genre|           developer|           publisher|          plateforms|
+--------------------+---------+---------+--------------+------------+----------------+--------------------+--------------------+--------------------+
|     forza-horizon-5|       92|      8.2|          4735| Nov 5, 2021| Auto Racing Sim|    Playground Games|   Xbox Game Studios|[PC, Xbox Series ...|
|           astro-bot|       94|      9.5|          2164| Sep 6, 2024|   3D Platformer|          Team Asobi|Sony Interactive ...|     [PlayStation 5]|
|     resident-evil-4|       93|      8.4|          6685|Mar 24, 2023|        Survival|              Capcom|              Capcom|[PlayStation 5, X...|
|        demons-souls|       92|      8.1|          4326|Nov 11, 2020|      Action RPG|     Bl

In [26]:
# Convert 'date' column in DateType format directly to ISO 8601
df = df.withColumn("date", to_date(col("date"), "MMM d, yyyy"))

df.show()

+--------------------+---------+---------+--------------+----------+----------------+--------------------+--------------------+--------------------+
|               title|metascore|userscore|reviews_number|      date|           genre|           developer|           publisher|          plateforms|
+--------------------+---------+---------+--------------+----------+----------------+--------------------+--------------------+--------------------+
|     forza-horizon-5|       92|      8.2|          4735|2021-11-05| Auto Racing Sim|    Playground Games|   Xbox Game Studios|[PC, Xbox Series ...|
|           astro-bot|       94|      9.5|          2164|2024-09-06|   3D Platformer|          Team Asobi|Sony Interactive ...|     [PlayStation 5]|
|     resident-evil-4|       93|      8.4|          6685|2023-03-24|        Survival|              Capcom|              Capcom|[PlayStation 5, X...|
|        demons-souls|       92|      8.1|          4326|2020-11-11|      Action RPG|     Bluepoint Games|

In [27]:
#We transform our string into the appropriate type
df = df.withColumn("metascore", col("metascore").cast("int"))
df = df.withColumn("reviews_number", col("reviews_number").cast("int"))
df = df.withColumn("userscore", col("userscore").cast("float"))

# Show DataFrame schema to check column types
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- metascore: integer (nullable = true)
 |-- userscore: float (nullable = true)
 |-- reviews_number: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- genre: string (nullable = true)
 |-- developer: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- plateforms: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [28]:
# Sort platforms in the 'platforms' column alphabetically
df = df.withColumn("plateforms", sort_array(col("plateforms")))

In [29]:
# Delete rows containing NULLs in any column
df = df.dropna()
df.show()

+--------------------+---------+---------+--------------+----------+----------------+--------------------+--------------------+--------------------+
|               title|metascore|userscore|reviews_number|      date|           genre|           developer|           publisher|          plateforms|
+--------------------+---------+---------+--------------+----------+----------------+--------------------+--------------------+--------------------+
|     forza-horizon-5|       92|      8.2|          4735|2021-11-05| Auto Racing Sim|    Playground Games|   Xbox Game Studios|[PC, Xbox One, Xb...|
|           astro-bot|       94|      9.5|          2164|2024-09-06|   3D Platformer|          Team Asobi|Sony Interactive ...|     [PlayStation 5]|
|     resident-evil-4|       93|      8.4|          6685|2023-03-24|        Survival|              Capcom|              Capcom|[PC, PlayStation ...|
|        demons-souls|       92|      8.1|          4326|2020-11-11|      Action RPG|     Bluepoint Games|

In [30]:
total_jeux_traites = df.count()
total_jeux = len(titles)
print("total games :",total_jeux)
print("total games processed:",total_jeux_traites)
print(int((total_jeux_traites/total_jeux)*100),"% games were kept after transformation")

total games : 2948
total games processed: 2944
99 % games were kept after transformation


In [31]:
liste_titre_jeux_traite = df.select("title").rdd.flatMap(lambda x: x).collect()

diff1 = [item for item in titles if item not in liste_titre_jeux_traite]
print("In titles but not in the final df:", diff1)

In titles but not in the final df: ['rose-and-camellia-collection', 'the-legend-of-zelda-echoes-of-wisdom', 'super-mario-party-jamboree', 'silent-hill-2']


In [32]:
bucket_name = "videogames-project"

# Get the day
today = datetime.today().date()

# Format the date as 'YYYY-MM-DD'
formatted_date = today.strftime('%Y-%m-%d')

path = f"s3a://{bucket_name}/dataset-{today}/" 

df.write.mode("overwrite").parquet(path)